In [ ]:
from portfolio_optimisation.classes.yahoo_parser import SP500Parser
yahoo_parser = SP500Parser()


start = '2010-01-01'
end = '2024-01-01'
data = yahoo_parser.download_sp500_data(start, end)

[                       1%%                      ]  6 of 503 completed

KeyboardInterrupt: 

[*********             18%%                      ]  93 of 503 completed

In [ ]:
data.to_csv('/Users/sprilut/workspace/hse/Portfolio_Optimisation/datasets/sp500_historical_data.csv')

In [ ]:
data.head()

,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2010-01-04,A,20.154915,22.389128,22.625179,22.267525,22.453505,3815561.0
2010-01-05,A,19.935982,22.145924,22.331903,22.002861,22.324751,4186031.0
2010-01-06,A,19.865147,22.067240,22.174536,22.002861,22.067240,3243779.0
2010-01-07,A,19.839396,22.038628,22.045780,21.816881,22.017166,3095172.0
2010-01-08,A,19.832952,22.031473,22.067240,21.745352,21.917025,3733918.0


In [1]:
import pandas as pd
data = pd.read_csv('/Users/sprilut/workspace/hse/Portfolio_Optimisation/datasets/sp500_historical_data.csv', index_col=0)
import pandas as pd
import numpy as np

hard_coded_portfolio = {
    'AAPL': 0.07,
    'MSFT': 0.065,
    'AMZN': 0.032,
    'NVDA': 0.028,
    'GOOGL': 0.021,
    'TSLA': 0.019,
    'GOOG': 0.018,
    'BRK.B': 0.018,
    'META': 0.018,
    'UNH': 0.013,
    'XOM': 0.013,
    'LLY': 0.012,
    'JPM': 0.012,
    'JNJ': 0.011,
    'V': 0.011,
    'PG': 0.01,
    'MA': 0.009,
    'AVGO': 0.009,
    'HD': 0.009,
    'CVX': 0.008,
    'MRK': 0.007,
    'ABBV': 0.007,
    'COST': 0.007,
    'PEP': 0.007,
    'ADBE': 0.007
}
best_25_tickers = list(hard_coded_portfolio.keys())
best_25_historical = data[data['Symbol'].isin(best_25_tickers)]

/var/folders/8v/ps3k66h11hjf_56rkxfbt3tm0000gn/T/ipykernel_1143/2585077090.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [21]:
import pandas as pd
import pandas as pd
from ta.trend import MACD
# Assuming 'data' is your DataFrame and it includes a 'Close' column

# Assuming 'data' is your DataFrame and it includes 'StockID' and 'Close' columns
# First, ensure your MACD and RSI functions are defined

def apply_features(group):
    group.index = pd.to_datetime(group.index)

    # Compute lag features, moving averages, etc., for the group
    for lag in range(1, 4):
        group[f'lag_{lag}'] = group['Close'].shift(lag)
    group['weekly_return'] = group['Close'].pct_change(5)
    group['5_day_MA'] = group['Close'].rolling(window=5).mean()
    group['20_day_MA'] = group['Close'].rolling(window=20).mean()
    group['5_day_volatility'] = group['Close'].rolling(window=5).std()
    group['momentum'] = group['Close'] - group['Close'].shift(1)
    
    # MACD, ensuring you handle NaNs as per your strategy

    macd = MACD(close=group['Close'], window_slow=26, window_fast=12, window_sign=9)
    group['MACD'] = macd.macd()
    group['MACD_signal'] = macd.macd_signal()
    group['MACD_histogram'] = macd.macd_diff()

    # Adjusting for multi-stock data: adding week_of_year and month
    group['week_of_year'] = group.index.isocalendar().week
    group['month'] = group.index.month
    
    return group.dropna()  # Optionally drop NaNs

# Apply the function to each group and recombine
data_grouped = best_25_historical.groupby('Symbol').apply(apply_features)


/var/folders/8v/ps3k66h11hjf_56rkxfbt3tm0000gn/T/ipykernel_1143/1829392717.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data_grouped = best_25_historical.groupby('Symbol').apply(apply_features)


In [62]:

# Initialize the random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)

# Fit the model on the training data
rf.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_rf = rf.predict(X_test)

# Calculate the mean squared error
evaluate_model(y_test, y_pred_rf, 'Random Forest')

Mean Absolute Error Random Forest: 0.01537962885924607
Root Mean Squared Error Random Forest: 0.023138437669156766
R-squared Score Random Forest: 0.679208612685972
Mean Squared Error Random Forest: 0.0005353872977694528


(0.01537962885924607,
 0.023138437669156766,
 0.679208612685972,
 0.0005353872977694528)

In [63]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the random forest regressor
rf = RandomForestRegressor(random_state=42)

# Initialize the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the grid search on the training data
grid_search.fit(data_groupd_scaled, data_grouped[target])

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# y_pred_rf_grid = grid_search.predict(X_test)

# evaluate_model(y_test, y_pred_rf_grid, 'Random Forest')

/Users/sprilut/Library/Caches/pypoetry/virtualenvs/portfolio-optimisation-qXJsKVfj-py3.11/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [55]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Initialize the gradient boosting regressor
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the model on the training data
gb.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_gb = gb.predict(X_test)

# Calculate the mean squared error
evaluate_model(y_test, y_pred_gb)

Mean Absolute Error : 0.018700536661704282
Root Mean Squared Error : 0.02724358461322014
R-squared Score : 0.5552836092820506
Mean Squared Error : 0.000742212902577685


(0.018700536661704282,
 0.02724358461322014,
 0.5552836092820506,
 0.000742212902577685)

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

# Define the degree of the polynomial features
degree = 2

# Create polynomial features
poly_features = PolynomialFeatures(degree=degree)
X_train_poly = poly_features.fit_transform(X_train)
X_test_poly = poly_features.transform(X_test)

# Initialize the linear regression model
linear_regression = LinearRegression()

# Fit the model on the polynomial features
linear_regression.fit(X_train_poly, y_train)

# Predict the target variable for the test data
y_pred_poly = linear_regression.predict(X_test_poly)

# Calculate the mean squared error
evaluate_model(y_test, y_pred_poly)


Mean Absolute Error : 0.017119825479023718
Root Mean Squared Error : 0.02745169933505419
R-squared Score : 0.5484632489365279
Mean Squared Error : 0.0007535957963822146


(0.017119825479023718,
 0.02745169933505419,
 0.5484632489365279,
 0.0007535957963822146)

In [57]:
from sklearn.ensemble import BaggingRegressor

# Initialize the base regressor
base_regressor = RandomForestRegressor()

# Initialize the bagging regressor
bagging_regressor = BaggingRegressor(estimator=base_regressor, n_estimators=10, random_state=42, n_jobs=-1)

# Fit the model on the training data
bagging_regressor.fit(X_train, y_train)

# Predict the target variable for the test data
y_pred_br = bagging_regressor.predict(X_test)

# Calculate the mean squared error
evaluate_model(y_test, y_pred_br)


Mean Absolute Error : 0.015396696857450497
Root Mean Squared Error : 0.023101946120431732
R-squared Score : 0.6802196527418847
Mean Squared Error : 0.0005336999145513307


(0.015396696857450497,
 0.023101946120431732,
 0.6802196527418847,
 0.0005336999145513307)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV

# Define the base regressor
base_regressor = RandomForestRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_samples': [0.5, 0.7, 0.9],
    'max_features': [0.5, 0.7, 0.9]
}

# Initialize the bagging regressor
bagging_regressor = BaggingRegressor(estimator=base_regressor, random_state=42)

# Initialize the grid search
grid_search = GridSearchCV(estimator=bagging_regressor, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

# Fit the grid search on the training data
grid_search.fit(data_groupd_scaled, data_grouped[target])

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Print the best parameters and best score
print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Predict the target variable for the test data
y_pred_bagging = grid_search.predict(X_test)

# Calculate the mean squared error
evaluate_model(y_test, y_pred_bagging, 'Bagging')


TypeError: BaggingRegressor.__init__() got an unexpected keyword argument 'base_estimator'

In [58]:
from sklearn.metrics import mean_absolute_error
from pickle import dump
errors = pd.DataFrame(columns=['ticker', 'mae', 'rmse', 'r2', 'mse'])
for ticker in data_grouped['Symbol'].unique():
    # Filter the data for the current ticker
    ticker_data = data_grouped[data_grouped['Symbol'] == ticker]
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(ticker_data[features], ticker_data[target], test_size=0.2, random_state=42)
    
    # Initialize the random forest regressor
    rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    
    # Fit the model on the training data
    rf.fit(X_train, y_train)
    
    # Predict the target variable for the test data
    y_pred = rf.predict(X_test)
    
    # Store the mean error for the current ticker
    with open(f'{ticker}_model.pkl', 'wb') as file:
        dump(rf, file)
    mae, rmse, r2, mse = evaluate_model(y_test, y_pred, ticker)
    errors = pd.concat([errors, pd.DataFrame({'ticker': ticker, 'mae': mae, 'rmse':rmse, 'r2': r2, 'mse': mse }, index=[len(errors.index) + 1])], ignore_index=True)

Mean Absolute Error AAPL: 0.015901832258997713
Root Mean Squared Error AAPL: 0.022113126726145106
R-squared Score AAPL: 0.6665172174333607
Mean Squared Error AAPL: 0.0004889903736065529


/var/folders/8v/ps3k66h11hjf_56rkxfbt3tm0000gn/T/ipykernel_1143/506980254.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  errors = pd.concat([errors, pd.DataFrame({'ticker': ticker, 'mae': mae, 'rmse':rmse, 'r2': r2, 'mse': mse }, index=[len(errors.index) + 1])], ignore_index=True)


Mean Absolute Error ABBV: 0.015443613212377747
Root Mean Squared Error ABBV: 0.0210653234810721
R-squared Score ABBV: 0.6442975097826229
Mean Squared Error ABBV: 0.0004437478533622075
Mean Absolute Error ADBE: 0.01838908636145741
Root Mean Squared Error ADBE: 0.026167353268725536
R-squared Score ADBE: 0.5961328177776118
Mean Squared Error ADBE: 0.000684730377090281
Mean Absolute Error AMZN: 0.021567158240305657
Root Mean Squared Error AMZN: 0.030648400066579568
R-squared Score AMZN: 0.5282349606045345
Mean Squared Error AMZN: 0.0009393244266411145
Mean Absolute Error AVGO: 0.021051938216730553
Root Mean Squared Error AVGO: 0.02805741850553021
R-squared Score AVGO: 0.6225893971616129
Mean Squared Error AVGO: 0.000787218733194469
Mean Absolute Error COST: 0.011364677290434761
Root Mean Squared Error COST: 0.016484461322204096
R-squared Score COST: 0.5659051235233616
Mean Squared Error COST: 0.0002717374650832428
Mean Absolute Error CVX: 0.01458178310157175
Root Mean Squared Error CVX: 0.

In [59]:
errors.describe()

,mae,rmse,r2,mse
count,24.000000,24.000000,24.000000,24.000000
mean,0.016292,0.023062,0.625471,0.000613
std,0.006057,0.009228,0.043102,0.000631
min,0.009005,0.012373,0.528235,0.000153
25%,0.013482,0.018861,0.598761,0.000356
50%,0.015013,0.021051,0.634107,0.000443
75%,0.016696,0.024049,0.657418,0.000580
max,0.037291,0.057516,0.684665,0.003308


In [48]:
apple_ts = data_grouped[data_grouped['Symbol'] == 'AAPL']
predictions = rf.predict(apple_ts[features])

from statsmodels.tsa.arima.model import ARIMA

# Assuming 'df' is your DataFrame with the datetime index and one column, e.g., 'Close'
series = apple_ts['weekly_return']  # Adjust column name as necessary

# Fit the ARIMA model
# Note: You may need to adjust the order (p,d,q) based on your specific data
model = ARIMA(series, order=(5,1,0))  # Example order, adjust based on AIC/BIC or domain knowledge
model_fit = model.fit()

# Make in-sample prediction
arima_predictions = model_fit.predict(start=series.index[0], end=series.index[-1], typ='levels')


/Users/sprilut/Library/Caches/pypoetry/virtualenvs/portfolio-optimisation-qXJsKVfj-py3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sprilut/Library/Caches/pypoetry/virtualenvs/portfolio-optimisation-qXJsKVfj-py3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

/Users/sprilut/Library/Caches/pypoetry/virtualenvs/portfolio-optimisation-qXJsKVfj-py3.11/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



In [49]:
# get only date index in the multi index
apple_ts = apple_ts.reset_index(level=0, drop=True)

In [53]:
import matplotlib.pyplot as plt
shifted_predictions = arima_predictions.shift(-1)
combined_df = pd.DataFrame({'ARIMA': shifted_predictions.values, 'Machine Learning': predictions, 'Real': apple_ts['weekly_return']}, index=apple_ts.index)
pd.options.plotting.backend = "plotly"
combined_df.plot()

In [57]:
def qualitySMAPE(x,y):
    # Symmetric mean absolute percentage error
    # x,y - pandas structures
    # x - real values
    # y - forecasts
    qlt = (2*(x-y).abs()/(x+y)).replace([np.inf, -np.inf], np.nan)
    return qlt.mean() , qlt
def calculate_smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_pred - y_true) / denominator
    diff[denominator == 0] = 0.0  # handle the case where both actuals and forecasts are zero
    return 100 * np.nanmean(diff)
print(f'Arima smape: {calculate_smape(apple_ts["weekly_return"], shifted_predictions)}')
print(f'ML smape: {calculate_smape(apple_ts["weekly_return"], predictions)}')

Arima smape: 54.03492279768201
ML smape: 49.18387005549422


In [55]:
feature_importances = pd.DataFrame(rf.feature_importances_, index=features, columns=['importance']).sort_values('importance', ascending=False)
feature_importances.plot(kind='bar')